# Modern Approaches to Struct Aggregation in Databricks

## Working with Nested Healthcare Data: Struct vs VARIANT Approaches

This notebook demonstrates different approaches for working with nested data in Databricks, comparing traditional struct approaches with modern VARIANT data types. We'll explore both deprecated and modern syntax for each approach.

### 📋 What You'll Learn:
- **Struct Approach**: Deprecated `LATERAL VIEW` vs modern table-valued functions
- **VARIANT Approach**: Modern schema-less data handling (Databricks Runtime 15.3+)
- **Higher-Order Functions**: Efficient aggregation without row explosion
- **Performance Comparison**: When to use each approach

### 🏥 Use Case: Healthcare Claims Processing
We'll work with a realistic nested data structure representing medical claims with header information and detailed line items.

### 🔄 Flow Overview:
1. **Struct Approach** - Data setup, deprecated querying, and modern querying
2. **VARIANT Approach** - Data setup, basic querying, and array explosion
3. **Higher-Order Functions** - Efficient aggregation for both approaches
4. **Comparison & Recommendations** - When to use each approach


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Initialize Spark session
spark = SparkSession.builder.appName("BlogValidationTest").getOrCreate()


## 1. Struct Approach

Let's start with the traditional struct approach using explicit schemas. This is the most common way to handle nested data in Databricks.

Each claim has:
- **Header information**: claim ID, line of business, total charges
- **Detail array**: individual charge line items with amounts and units


### 1.1 Struct Data Setup

Let's create the data using traditional structured types with explicit schemas:


In [2]:
# Create the nested data structure matching the blog screenshot
medicaid_data = [
    {
        "claimHeader": {
            "claimId": "ABC123456789",
            "lineOfBusiness": "Medicaid",
            "totalCharges": 3.25
        },
        "claimDetail": [
            {"chargeAmount": 1.25, "units": 1.00},
            {"chargeAmount": 2.00, "units": 1.00}
        ]
    }
]

# Define the schema
schema = StructType([
    StructField("claimHeader", StructType([
        StructField("claimId", StringType(), True),
        StructField("lineOfBusiness", StringType(), True),
        StructField("totalCharges", DoubleType(), True)
    ]), True),
    StructField("claimDetail", ArrayType(StructType([
        StructField("chargeAmount", DoubleType(), True),
        StructField("units", DoubleType(), True)
    ])), True)
])

# Create DataFrame
myClaimsTable = spark.createDataFrame(medicaid_data, schema)
myClaimsTable.createOrReplaceTempView("myClaimsTable")

print("Created test data:")
myClaimsTable.show(truncate=False)
myClaimsTable.printSchema()


Created test data:
+------------------------------+-------------------------+
|claimHeader                   |claimDetail              |
+------------------------------+-------------------------+
|{ABC123456789, Medicaid, 3.25}|[{1.25, 1.0}, {2.0, 1.0}]|
+------------------------------+-------------------------+

root
 |-- claimHeader: struct (nullable = true)
 |    |-- claimId: string (nullable = true)
 |    |-- lineOfBusiness: string (nullable = true)
 |    |-- totalCharges: double (nullable = true)
 |-- claimDetail: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- chargeAmount: double (nullable = true)
 |    |    |-- units: double (nullable = true)



### 1.2 Struct Querying: Deprecated Approach ❌

⚠️ **Important**: `LATERAL VIEW` is deprecated in Databricks Runtime 12.2+ but still works. This shows the old syntax for comparison.


In [3]:
%sql
-- DEPRECATED SYNTAX (still works but not recommended)
SELECT 
    claimHeader.claimId,
    claimHeader.lineOfBusiness,
    d.chargeAmount,
    d.units
FROM myClaimsTable
LATERAL VIEW explode(claimDetail) AS d


,claimId,lineOfBusiness,chargeAmount,units
0,ABC123456789,Medicaid,1.25,1.0
1,ABC123456789,Medicaid,2.00,1.0


### 1.3 Struct Querying: Modern Approach ✅

**Recommended for Databricks Runtime 12.2+**

This is the modern way to explode arrays using table-valued functions as table references:


In [4]:
%sql
-- MODERN SYNTAX (Recommended for Runtime 12.2+)
SELECT 
    claimHeader.claimId,
    claimHeader.lineOfBusiness,
    d.col.chargeAmount,
    d.col.units
FROM myClaimsTable,
    LATERAL explode(claimDetail) AS d


,claimId,lineOfBusiness,chargeAmount,units
0,ABC123456789,Medicaid,1.25,1.0
1,ABC123456789,Medicaid,2.00,1.0


## 2. VARIANT Approach

Now let's explore the modern VARIANT data type (Databricks Runtime 15.3+), which is more flexible for semi-structured data:


### 2.1 VARIANT Data Setup

Let's create the same data using the modern VARIANT data type:


In [5]:
# Create the same data using VARIANT type (schema-less)
claims_json_data = '''
{
  "claimHeader": {
    "claimId": "ABC123456789",
    "lineOfBusiness": "Medicaid", 
    "totalCharges": 3.25
  },
  "claimDetail": [
    {"chargeAmount": 1.25, "units": 1.00},
    {"chargeAmount": 2.00, "units": 1.00}
  ]
}
'''

# Create DataFrame with VARIANT column using parse_json
df_variant = spark.sql(f"""
  SELECT parse_json('{claims_json_data}') as claim_data
""")

# Create a view for easier querying
df_variant.createOrReplaceTempView("myClaimsTableVariant")

# Display the VARIANT data
print("Created VARIANT data:")
df_variant.show(truncate=False)
df_variant.printSchema()


Created VARIANT data:


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|claim_data                                                                                                                                                             |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"claimDetail":[{"chargeAmount":1.25,"units":1},{"chargeAmount":2,"units":1}],"claimHeader":{"claimId":"ABC123456789","lineOfBusiness":"Medicaid","totalCharges":3.25}}|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+

root
 |-- claim_data: variant (nullable = false)



### 2.2 VARIANT Querying: Basic Field Access

VARIANT data uses different syntax for field access and querying. Let's explore the modern approach:


VARIANT data uses different syntax for field access. Let's see how to query the same fields:


In [6]:
%sql
-- VARIANT field access uses colon syntax
SELECT 
    claim_data:claimHeader.claimId::string AS claimId,
    claim_data:claimHeader.lineOfBusiness::string AS lineOfBusiness,
    claim_data:claimHeader.totalCharges::double AS totalCharges
FROM myClaimsTableVariant


,claimId,lineOfBusiness,totalCharges
0,ABC123456789,Medicaid,3.25


### 2.3 VARIANT Querying: Exploding Arrays

For VARIANT data, we use `variant_explode()` instead of `explode()`:


In [ ]:
%sql
-- VARIANT array explosion
SELECT 
    claim_data:claimHeader.claimId::string AS claimId,
    claim_data:claimHeader.lineOfBusiness::string AS lineOfBusiness,
    detail.value:chargeAmount::double AS chargeAmount,
    detail.value:units::double AS units
FROM myClaimsTableVariant,
    LATERAL variant_explode(claim_data:claimDetail) AS detail


## 3. Higher-Order Functions: The Most Efficient Approach 🚀

Higher-order functions provide the most efficient way to work with nested data by avoiding row explosion. Let's explore these powerful functions for both struct and VARIANT data.


### 3.1 Struct Higher-Order Functions

Instead of exploding rows, we can use `aggregate()` to calculate totals directly:


In [7]:
%sql
-- Struct Higher-Order Function Example
SELECT 
    claimHeader.claimId,
    claimHeader.lineOfBusiness,
    claimHeader.totalCharges as original_totalCharges,
    aggregate(claimDetail, CAST(0.0 AS DOUBLE), (acc, detail) -> acc + detail.chargeAmount * detail.units) as calculated_totalCharges
FROM myClaimsTable


,claimId,lineOfBusiness,original_totalCharges,calculated_totalCharges
0,ABC123456789,Medicaid,3.25,3.25


### 3.2 VARIANT Higher-Order Functions

For VARIANT data, we need to use `variant_explode()` with traditional aggregation since `aggregate()` doesn't work directly with VARIANT data:


In [8]:
%sql
-- VARIANT Aggregation Example (Correct Approach)
-- Note: aggregate() doesn't work directly with VARIANT data
-- Use variant_explode() with traditional aggregation instead
SELECT 
    claim_data:claimHeader.claimId::string AS claimId,
    claim_data:claimHeader.lineOfBusiness::string AS lineOfBusiness,
    claim_data:claimHeader.totalCharges::double AS originalTotal,
    SUM(detail.value:chargeAmount::double * detail.value:units::double) AS calculatedTotal
FROM myClaimsTableVariant,
    LATERAL variant_explode(claim_data:claimDetail) AS detail
GROUP BY 
    claim_data:claimHeader.claimId::string,
    claim_data:claimHeader.lineOfBusiness::string,
    claim_data:claimHeader.totalCharges::double


,claimId,lineOfBusiness,originalTotal,calculatedTotal
0,ABC123456789,Medicaid,3.25,3.25


## 4. Approach Comparison & Recommendations

### 📊 Struct vs VARIANT Comparison

| Aspect | Struct | VARIANT |
|--------|--------|---------|
| **Schema** | Explicit, defined at creation | Implicit, inferred from data |
| **Type Safety** | Strong typing, compile-time checks | Runtime type checking |
| **Performance** | Faster for known structures | Slightly slower due to type inference |
| **Flexibility** | Fixed schema, requires migration | Schema evolution friendly |
| **Query Syntax** | Dot notation (`field.subfield`) | Colon notation (`field:subfield`) |
| **Array Functions** | `explode()` | `variant_explode()` |
| **Higher-Order Functions** | Direct `aggregate()` support | Requires `variant_explode()` + traditional aggregation |
| **Best For** | Known, stable schemas | Semi-structured, evolving data |

### 🎯 Best Practices & Recommendations

1. **For Structured Data**: Use struct with higher-order functions (`aggregate`, `transform`, `filter`)
2. **For Semi-Structured/JSON**: Use VARIANT data type with `variant_explode()` (Runtime 15.3+)
3. **For Migration**: Replace `LATERAL VIEW` with modern table-valued function syntax
4. **For Performance**: Higher-order functions avoid row explosion and are most efficient
5. **For Schema Evolution**: VARIANT provides flexibility for changing data structures
6. **For Readability**: Choose the approach your team is most comfortable with

### 📚 References

- [Databricks LATERAL VIEW Documentation](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/sql-ref-syntax-qry-select-lateral-view)
- [Modern Table Reference Syntax](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/sql-ref-syntax-qry-select-table-reference)
- [VARIANT Data Type Documentation](https://learn.microsoft.com/en-us/azure/databricks/semi-structured/variant)
- [Higher-Order Functions in Spark SQL](https://spark.apache.org/docs/latest/sql-ref-functions-builtin.html#higher-order-functions)

---

**💡 Pro Tips**: 
- **For structured data**: Start with higher-order functions - they're most performant
- **For JSON/APIs**: Use VARIANT data type for flexibility and easier schema evolution  
- **For migration**: Move from LATERAL VIEW → modern LATERAL → higher-order functions
- **Performance matters**: Higher-order functions > modern LATERAL > VARIANT (best)
